In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
test = pd.read_csv("test_data (1).csv")

In [2]:
train_df = pd.read_csv('training_data.csv')
train_df.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50
0,8.536145,-1.535413,0.718888,-2.099149,-0.442842,-0.598978,-1.642574,0.207755,0.760642,0.575874,...,0.361866,1.793098,-0.631287,-0.061751,0.511049,0.488754,-0.612772,-0.471045,-1.139781,-0.260773
1,4.808344,-1.734609,0.551981,-2.147673,-1.552944,1.514910,-1.143972,0.737594,1.321243,-0.261684,...,-0.677985,-0.165679,0.065405,0.137162,1.258197,-0.120834,-1.564834,-0.242565,-0.001827,1.187453
2,-1.530427,0.097257,0.107634,-0.194222,0.335454,-0.408199,0.133265,0.706179,0.394971,-0.437116,...,1.108801,0.333791,0.282055,-1.086294,-0.115354,0.257857,-0.088838,-0.751231,1.450609,0.290593
3,-0.428243,-0.067702,0.557836,0.700848,-1.121376,1.722274,0.613525,0.700909,-0.417976,1.069749,...,0.692511,-0.350990,0.624558,0.434520,-0.367409,-1.144681,-0.136524,-0.557214,0.416303,0.484495
4,0.566694,0.488729,0.211483,0.568389,0.646837,0.163868,-0.002152,0.125137,0.493571,1.705451,...,-0.000605,1.075280,0.182281,-1.138458,0.106092,0.544640,-0.383487,-0.425773,2.667647,-0.050748


### K-Fold Find Best # of Params

In [3]:
df = pd.DataFrame(columns = ['k', 'ObjVal', 'sse_train', 'sse_validation', 'constant'])#.set_index('k')
ndata = len(train_df)
folds = 10
total_rows = [i for i in range(250)]
M = 100
time_limit = 12*60*60

for k in range(5, 51, 5):
    shuffle = np.random.choice(range(ndata),size=ndata,replace=False)
    sse_validation = 0
    sse_train = 0
    for j in range(folds):
        data_size = int(ndata/folds)
        val = total_rows[j*(data_size):(j+1)*data_size]
        train_rows = list(set(total_rows) - set(val))
        validation_set = train_df.iloc[shuffle[val],:]
        validation_x = train_df.iloc[shuffle[val],:].drop(columns = 'y')
        validation_y = train_df.iloc[shuffle[val],:].filter(like = 'y')
        validation_y = validation_y.to_numpy().T[0]
        train_set = train_df.iloc[shuffle[train_rows],:]
        train_x = train_set.drop(columns = 'y')
        train_y = train_set.filter(like = 'y')

        #setting X's
        X = train_x
        m = X.shape[1]
        idx = 0
        n = X.shape[0]
        new_col = [1]* n  
        X.insert(loc=idx, column='X0', value=new_col)
        x_arr = X.to_numpy()

        #setting up Y's
        Y = train_y
        y_arr = Y.to_numpy()
        y_mat = y_arr.T[0]

        #Q matrix
        rows = 2*m+1
        cols = 2*m+1
        q_matrix = np.asarray([[0]*cols]*rows, dtype = "float") #needed to make python interpret as float instead of int
        q_matrix[:m+1,:m+1] = x_arr.T @ x_arr

        #setting up linear term
        linear_term = np.zeros(2*m+1)
        linear_term[:m+1] = (-2)* y_arr.T @ x_arr
        
        #A matrix
        A = np.zeros((2*m+1,2*m+1)) # row 0 is all 1s already
        A[0,-m:] = [1]*m # constraint on z
        A[:m,1:m+1] = np.eye(m)
        A[m:-1,1:m+1] = np.eye(m)
        A[:m,m+1:2*m+1] = np.eye(m)*(-M)
        A[m:-1,m+1:2*m+1] = np.eye(m)*(M)
        A[-1, m+1:] = 1

        # setting up b
        b = [0]*(2*m+1)
        b[-1] = k
        b_arr = np.array(b)

        #setting sense
        sense = ['<']*(m) + ['>']*(m) + ['<']
        sense_arr = np.array(sense)
        model = gp.Model()
        mod_x = model.addMVar(2*m+1,vtype=np.array(['C']*(m+1) + ['B']*m), lb=np.array([np.NINF]+[-M]*(m)+ [np.NINF]*m))
        mod_con = model.addMConstr(A, mod_x, sense_arr, b_arr)
        
        #setting obj
        model.setMObjective(q_matrix,linear_term,0,sense=gp.GRB.MINIMIZE)
        model.Params.OutputFlag = 0
        model.setParam('IntFeasTol', 1e-9)
        model.setParam('TimeLimit', time_limit)
        model.optimize()
        my_coeffs = mod_x.x[:m+1] #past m+1 is the z
        
        #validation set
        rows = validation_x.shape[0]
        cols = m+1
        val_matrix = np.array([[1]*rows], dtype = "float") 
        val_matrix = np.vstack((val_matrix, validation_x.T)).T
        val_matrix[-1,0] = 1

    #calculate the SSE 
        sse_validation += (val_matrix @ my_coeffs - validation_y).T @ (val_matrix @ my_coeffs - validation_y)
        sse_train = (x_arr @ my_coeffs - y_mat) @ np.transpose(x_arr @ my_coeffs - y_mat)
        constant = y_mat.T @ y_mat
    df.loc[len(df.index)]= [k, model.ObjVal, sse_train, sse_validation, constant]
    display(df)
    print(my_coeffs)
    df.to_csv("TestOpt3.csv")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


,k,ObjVal,sse_train,sse_validation,constant
0,5.0,-4094.070396,771.410158,912.902551,4865.480554


[ 1.06576853  0.          0.          0.          0.          0.
  0.          0.          0.         -2.42458695  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         -1.67008761
  0.          0.         -1.60672304  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.912944    0.          0.7890679
  0.          0.          0.        ]


,k,ObjVal,sse_train,sse_validation,constant
0,5.0,-4094.070396,771.410158,912.902551,4865.480554
1,10.0,-4229.913569,527.063782,677.893916,4756.977351


[ 0.96362703  0.          0.          0.          0.          0.
  0.          0.          0.         -2.26096553  0.          0.
  0.          0.          0.         -0.59345415  0.          0.
  0.          0.          0.          0.         -0.29875027 -1.50146728
  0.8829761   0.         -1.40975206  0.          0.          0.
  0.          0.          0.          0.          0.34327789  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.77252451  0.          0.88444937
 -0.23396681  0.          0.        ]


...

[ 0.92313848  0.          0.14200184  0.3047868   0.38974518 -0.10283925
 -0.28190652  0.58967983  0.         -2.76450361  0.57130112  0.
 -0.42248935  0.66201518  0.46162396 -0.38412229 -0.25185444  0.63104113
  0.62911894  0.73336699  0.87707045  0.14059389 -0.51084028 -1.29987497
  0.70875168  0.         -1.60897489  0.         -0.43955235  0.52640415
  0.          0.          0.8543127   0.          1.22170826 -0.50415849
  0.34133078  0.28387864  0.         -0.98370559  0.52959718 -0.73920809
  0.37400849 -0.40862513 -0.1238288   1.83750039  0.          1.08568198
  0.16727361  0.58775895 -0.38760938]


,k,ObjVal,sse_train,sse_validation,constant
0,5.0,-4094.070396,771.410158,912.902551,4865.480554
1,10.0,-4229.913569,527.063782,677.893916,4756.977351
2,15.0,-4269.283437,520.795131,793.194861,4790.078568
3,20.0,-4385.809111,504.144788,803.385089,4889.953900
4,25.0,-4247.765658,482.445286,824.701244,4730.210944
5,30.0,-4457.222335,457.692330,829.805498,4914.914665
6,35.0,-4335.256590,469.069260,835.320639,4804.325851
7,40.0,-4324.228656,478.785610,844.381711,4803.014266
8,45.0,-4289.754537,440.766149,847.698211,4730.520686


[ 1.01361284  0.          0.0539473   0.20732274  0.19146202  0.
 -0.18484289  0.29699316  0.         -2.57010519  0.31594516 -0.0444334
 -0.19017393  0.529947    0.35016486 -0.17285046 -0.31401566  0.44731705
  0.47448922  0.6435341   0.7031801  -0.10532711 -0.3427332  -1.23817181
  0.72121726 -0.09184615 -1.58248228  0.07576688 -0.20578699  0.44114462
  0.06813506 -0.15823341  0.61000388 -0.20730967  1.00257122 -0.26568149
  0.18602476  0.35939075  0.         -0.90396968  0.48390021 -0.72364953
  0.13547328 -0.19214194  0.          1.64138869 -0.10491841  0.96541985
  0.          0.55654686 -0.12685927]


,k,ObjVal,sse_train,sse_validation,constant
0,5.0,-4094.070396,771.410158,912.902551,4865.480554
1,10.0,-4229.913569,527.063782,677.893916,4756.977351
2,15.0,-4269.283437,520.795131,793.194861,4790.078568
3,20.0,-4385.809111,504.144788,803.385089,4889.953900
4,25.0,-4247.765658,482.445286,824.701244,4730.210944
5,30.0,-4457.222335,457.692330,829.805498,4914.914665
6,35.0,-4335.256590,469.069260,835.320639,4804.325851
7,40.0,-4324.228656,478.785610,844.381711,4803.014266
8,45.0,-4289.754537,440.766149,847.698211,4730.520686
9,50.0,-4557.376393,456.083477,873.702721,5013.459870


[ 0.98998831  0.3072296   0.14115441  0.52820934  0.32484734 -0.01648325
 -0.0680678   0.74378808 -0.05821214 -2.9168856   0.76243011  0.43785125
 -0.48374507  0.66029535  0.40306938 -0.09650313 -0.11350528  0.53233162
  0.43546073  0.40196895  0.8607646   0.05036304 -0.31726946 -1.28187541
  0.92669039  0.19395411 -1.44778665  0.21380412 -0.10311409  0.25111139
  0.24231715 -0.17011136  0.81411832 -0.01921313  1.51703955 -0.52211119
  0.30365957  0.10143919 -0.23762095 -1.41599745  0.51736891 -0.41175474
  0.61057528 -0.60405234  0.1937842   1.78662671 -0.29294356  0.99899473
  0.06912033  0.2852322  -0.32444395]


# Fit on Entire Training Data with k = 10

In [4]:
M = 100
time_limit = 12*60*60
k = 10

train_x = train_df.iloc[0:,1:]
train_y = train_df['y']

#setting X's
X = train_x
m = X.shape[1]
idx = 0
n = X.shape[0]
new_col = [1]* n  
X.insert(loc=idx, column='X0', value=new_col)
x_arr = X.to_numpy()

#setting up Y's
Y = train_y
y_arr = Y.to_numpy()
y_mat = y_arr.T[0]

#Q matrix
rows = 2*m+1
cols = 2*m+1
q_matrix = np.asarray([[0]*cols]*rows, dtype = "float") #needed to make python interpret as float instead of int
q_matrix[:m+1,:m+1] = x_arr.T @ x_arr

#setting up linear term
linear_term = np.zeros(2*m+1)
linear_term[:m+1] = (-2)* y_arr.T @ x_arr

M = 100

#A matrix
A = np.zeros((2*m+1,2*m+1)) # row 0 is all 1s already
A[0,-m:] = [1]*m # constraint on z
A[:m,1:m+1] = np.eye(m)
A[m:-1,1:m+1] = np.eye(m)
A[:m,m+1:2*m+1] = np.eye(m)*(-M)
A[m:-1,m+1:2*m+1] = np.eye(m)*(M)
A[-1, m+1:] = 1

# setting up b
b = [0]*(2*m+1)
b[-1] = k
b_arr = np.array(b)

#setting sense
sense = ['<']*(m) + ['>']*(m) + ['<']
sense_arr = np.array(sense)
model = gp.Model()
mod_x = model.addMVar(2*m+1,vtype=np.array(['C']*(m+1) + ['B']*m), lb=np.array([np.NINF]+[-M]*(m)+ [np.NINF]*m))
mod_con = model.addMConstr(A, mod_x, sense_arr, b_arr)

#setting obj
model.setMObjective(q_matrix,linear_term,0,sense=gp.GRB.MINIMIZE)
model.Params.OutputFlag = 0
model.setParam('IntFeasTol', 1e-9)
model.setParam('TimeLimit', time_limit)
model.optimize()
my_coeffs = mod_x.x[:m+1] #past m+1 is the z

#calculate the SSE 
sse_train = (x_arr @ my_coeffs - y_mat) @ np.transpose(x_arr @ my_coeffs - y_mat)
#constant = y_mat.T @ y_mat
#print(my_coeffs, sse_train, model.objVal)

#testing
test = pd.read_csv('test_data (1).csv')
test_x = np.array(test.iloc[0:,1:])
test_y = np.array(test['y'])

n = test_x.shape[0] 
m = test_x.shape[1] 

rows = n
cols = m+1
test_matrix = np.array([[1]*rows], dtype = "float") 
test_matrix = np.vstack((test_matrix, test_x.T)).T
test_matrix[-1,0] = 1
test_matrix @ my_coeffs

print('The Testing MSE is:', mean_squared_error(test_y, test_matrix @ my_coeffs))

The Testing MSE is: 2.3365439645525257


# Lasso Regression

In [5]:
X_train = np.array(train_df.iloc[0:,1:])
X_test = np.array(test.iloc[0:,1:])
y_train = np.array(train_df['y'])
y_test = np.array(test['y'])

lasso = linear_model.LassoCV(cv=10).fit(X_train, y_train) #10 k-fold
y_predict = lasso.predict(X_test)

print(f'the alpha is {lasso.alpha_}')
print(f'the intercept is {lasso.intercept_}')
print(f'number of x variables is:', (lasso.coef_ != 0).sum(), f'and the coefficients are {lasso.coef_}')
print('Test MSE:', mean_squared_error(y_test, y_predict))

the alpha is 0.07638765995113514
the intercept is 1.0015522007348236
number of x variables is: 17 and the coefficients are [-0.         -0.          0.          0.         -0.          0.
 -0.         -0.         -2.16054765  0.         -0.05964031 -0.
 -0.         -0.         -0.41912484 -0.19325408  0.          0.
 -0.          0.          0.         -0.19517759 -1.36388738  0.7425965
 -0.         -1.30481574 -0.          0.          0.05798283  0.
 -0.          0.         -0.09737839  0.28341629  0.          0.
  0.          0.         -0.23157873  0.         -0.          0.
  0.          0.03078191  1.56362172 -0.02160033  0.69992447 -0.09289745
  0.          0.        ]
Test MSE: 2.3496347591605806
